In [ ]:
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
import sam3

np.random.seed(3)

def show_mask(mask, ax, random_color=False, borders = True):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask = mask.astype(np.uint8)
    mask_image =  mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    if borders:
        import cv2
        contours, _ = cv2.findContours(mask,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
        # Try to smooth contours
        contours = [cv2.approxPolyDP(contour, epsilon=0.01, closed=True) for contour in contours]
        mask_image = cv2.drawContours(mask_image, contours, -1, (1, 1, 1, 0.5), thickness=2) 
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0, 0, 0, 0), lw=2))    

def show_masks(image, masks, scores, point_coords=None, box_coords=None, input_labels=None, borders=True):
    for i, (mask, score) in enumerate(zip(masks, scores)):
        plt.figure(figsize=(10, 10))
        plt.imshow(image)
        show_mask(mask, plt.gca(), borders=borders)
        if point_coords is not None:
            assert input_labels is not None
            show_points(point_coords, input_labels, plt.gca())
        if box_coords is not None:
            # boxes
            show_box(box_coords, plt.gca())
        if len(scores) > 1:
            plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
        plt.axis('off')
        plt.show()

In [ ]:
from inference.models.sam3 import Sam3ForInteractiveImageSegmentation


model = Sam3ForInteractiveImageSegmentation(api_key="EQ8CBtIdyAMC4KZuflxx")



In [ ]:


model = Sam3ForInteractiveImageSegmentation(api_key="EQ8CBtIdyAMC4KZuflxx")



In [ ]:
from PIL import Image


image = Image.open("./car.png")
image2 = Image.open("./traffic.jpg")



In [ ]:
#without cache:
#import uuid
#model.embed_image(image2, image_id=str(uuid.uuid4()))


img_embed = model.embed_image(image)
img_embed


proced = model.preproc_image(image)
proced.shape




In [ ]:
from inference.core.entities.requests.sam2 import (
    Sam2EmbeddingRequest,
    Sam2InferenceRequest,
    Sam2Prompt,
    Sam2PromptSet,
    Sam2SegmentationRequest,
    Point,
    InferenceRequestImage
)

prompts = Sam2PromptSet(prompts=[
    Sam2Prompt(points=[Point(x=300, y=300, positive=True)])
])

masks, scores, low_resolution_logits  = model.segment_image(image=image, prompts=prompts)

In [ ]:

masks.shape

In [ ]:
input_point = np.array([[300, 300]])
input_label = np.array([1])
show_masks(image, masks, scores, point_coords=input_point, input_labels=input_label, borders=True)


In [ ]:
import numpy as np


np_image = np.array(image)

img = InferenceRequestImage(type="file", value="./car.png")

prompts = Sam2PromptSet(prompts=[
    Sam2Prompt(points=[Point(x=100, y=100, positive=True)])
])

request = Sam2SegmentationRequest(format="polygons", image=img, prompts=prompts)


result = model.infer_from_request(request)

In [ ]:
result

In [ ]:
from PIL import Image, ImageDraw
preds = data.get("predictions", [])
if preds:
    with Image.open(img_path).convert("RGBA") as base:
        overlay = Image.new("RGBA", base.size, (0, 0, 0, 0))
        draw = ImageDraw.Draw(overlay, "RGBA")

        # Simple palette
        colors = [
            (72, 146, 234, 96),  # blue
            (0, 238, 195, 96),  # teal
            (254, 78, 240, 96),  # magenta
            (244, 0, 78, 96),  # red
            (250, 114, 0, 96),  # orange
            (238, 238, 23, 96),  # yellow
            (144, 255, 0, 96),  # green
        ]

        def to_xy(points: List[List[float]]) -> List[Tuple[float, float]]:
            return [(float(x), float(y)) for x, y in points]

        for i, pred in enumerate(preds):
            color = colors[i % len(colors)]
            for poly in pred.get("masks", []):
                if len(poly) >= 3:
                    draw.polygon(
                        to_xy(poly), fill=color, outline=color[:3] + (180,)
                    )

        result = Image.alpha_composite(base, overlay).convert("RGB")
        out_path = Path(args.out)
        result.save(out_path, format="JPEG")
        print(f"Saved visualization to {out_path.resolve()}")
else:
    print("No predictions to visualize.")